<a href="https://colab.research.google.com/drive/1TT5M7GhKLLcJCLjkq4edUWxH_u7kZbMn?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In [ ]:
!pip install -qU google-generativeai

In [ ]:
import google.generativeai as genai
import re
import getpass

Get Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [ ]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google AI API key: ··········


In [ ]:
# Configure API
genai.configure(api_key=API_KEY)

In [ ]:
class ReActAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.tools = {}
        self.history = []

    def add_tool(self, name, func, description):
        self.tools[name] = {"func": func, "desc": description}

    def run(self, query, max_steps=5):
        print(f"\n🎯 Query: {query}\n")

        for step in range(max_steps):
            # Create prompt
            tools_desc = "\n".join([f"{n}: {t['desc']}" for n, t in self.tools.items()])
            prompt = f"""You are a ReAct agent. Follow this format:
            Thought: [your reasoning]
            Action: tool_name(param="value")
            OR
            Final Answer: [final response]

            Tools available:
            {tools_desc}

            Query: {query}

            History:
            {chr(10).join(self.history)}

            Response:"""

            # Get response
            response = self.model.generate_content(prompt).text
            print(f"Step {step+1}:\n{response}\n")

            # Check if done
            if "Final Answer:" in response:
                return response.split("Final Answer:")[-1].strip()

            # Execute action
            match = re.search(r'Action:\s*(\w+)\((.*?)\)', response)
            if match:
                tool_name = match.group(1)
                params_str = match.group(2)

                # Parse params
                params = {}
                for p in re.finditer(r'(\w+)=["\'](.*?)["\']', params_str):
                    params[p.group(1)] = p.group(2)

                # Execute tool
                if tool_name in self.tools:
                    result = self.tools[tool_name]["func"](**params)
                    print(f"🔧 Tool Result: {result}\n")
                    self.history.append(f"Action: {tool_name}, Result: {result}")

            self.history.append(response)

        return "Max steps reached"


# Define tools
def search(query):
    responses = {
        "python": "Python is a high-level programming language.",
        "gemini": "Gemini is Google's multimodal AI model.",
    }
    for key, val in responses.items():
        if key in query.lower():
            return val
    return f"Found general info about: {query}"

def calc(expression):
    try:
        return str(eval(expression))
    except:
        return "Error in calculation"

def weather(city):
    data = {"london": "15°C, Cloudy", "paris": "17°C, Sunny"}
    return data.get(city.lower(), "No data")


In [ ]:
# Usage
agent = ReActAgent()
agent.add_tool("search", search, "Search for information")
agent.add_tool("calc", calc, "Calculate math expressions")
agent.add_tool("weather", weather, "Get weather for a city")

# Run examples
result = agent.run("What is 25 * 4 + 10?")
print(f"✅ Final: {result}\n")

result = agent.run("Search about Python and tell me what it is")
print(f"✅ Final: {result}")


🎯 Query: What is 25 * 4 + 10?

Step 1:
Thought:I need to calculate the result of the given expression.
Action: calc(expression="25 * 4 + 10")


🔧 Tool Result: 110

Step 2:
Thought:The expression has already been calculated in the history.
Final Answer: 110


✅ Final: 110


🎯 Query: Search about Python and tell me what it is

Step 1:
Thought:I have already calculated the result of the mathematical expression using the calculator tool. Now I should provide the final answer, which is the result I obtained.
Final Answer: 110


✅ Final: 110
